# Building 3D Estimator for a Quadcopter

The aim of this project is to build a 3D estimator for a quadcopter using Extended Kalman Filter(EKF). The estimator build in this project uses the following sensors IMU Sensors( Accelerometers and Gyroscope), Gps, Magnetometer. The [starter code](https://github.com/udacity/FCND-Estimation-CPP) provide already contains the estimator implement partially and in this project the functions inside the estimator are built. This writeup disuss how each point in the project [rubic](https://review.udacity.com/#!/rubrics/1807/view) are satisfied.

## Implementation of the Estimator

The implementation strategy of the estimator is discussed in detailed in the lecture lesson __3D EKF and UKF__. The quadcopter has 12 states and the following table describes the state vector.

|State|Description|
|:-|:-|
|$x$| Position along x axis in the intertial frame|
|$y$| Position along y axis in the intertial frame|
|$z$| Position along x axis in the intertial frame|
|$\dot{x}$| Velocity along x axis in the intertial frame|
|$\dot{y}$| Velocity along y axis in the intertial frame|
|$\dot{z}$| Velocity along x axis in the intertial frame|
|$\theta$| Orientation of drone along x axis in intertial frame it is called as pitch|
|$\phi$| Orientation of drone along y axis in intertial frame it is called as roll|
|$\psi$| Orientation of drone along z axis in intertial frame it is called as yaw|
|p| Angular velocity of the drone along x axis in the body frame|
|q| Angular velocity of the drone along y axis in the body frame|
|r| Angular velocity of the drone along z axis in the body frame|

This implementation of the estimator does not have all the drone states in the EKF, onle the state $x,y,z,\dot{x},\dot{y},\dot{z},\psi$ are estimated using the EKF. The states p,q,r are directly measured from gyroscope. And the states pitch and roll ($\theta,\phi$) are estimated using a complementry filter based on the accelerometer and gyroscope measurements. Another important thing to note in this implementation is that the acclerometer data are used as control inputs the prediction step instead of measurement update step. The following table shows the different sensors and how they are used.

|Sensors| Usage|
|:-|:-|
|Accerometer(IMU)| Used in the prediction step to estimate $x,y,z,\dot{x},\dot{y},\dot{z}$ and used in the complementry filter for estimating $\theta,\phi$|
|Gyroscope(IMU)| Used to estimate p,q,r and used in the complementry filter to estimate pitch, roll and yaw angles|
|Magnetometer| Measures Yaw angle and it used in the update step of EKF|
|Gps|Measures $x,y,z,\dot{x},\dot{y},\dot{z}$ and it used in the update step of EKF|

The equations of the EKF for this implmentation is described in the [Estimation for Quadrotors](https://www.overleaf.com/read/vymfngphcccj#/54894644/) document.

## The Tasks 

The project is structure to complete the following tasks.Each task builds a part of the estimator and each task has a test scenario is present in simulator to evaluate the implemention.

 - [Step 1: Sensor Noise](#Step-1-Sensor-Noise)
 - [Step 2: Attitude Estimation](#Step-2-Attitude-Estimation)
 - [Step 3: Prediction Step](#Step-3-Prediction-Step)
 - [Step 4: Magnetometer Update](#Step-4-Magnetometer-Update)
 - [Step 5: Closed Loop + GPS Update](#Step-5-Closed-loop--GPS-Update)
 - [Step 6: Adding Your Controller](#Step-6-Adding-Your-Controller)
 
The following sections describes how each task is implemented and these task form the rubic points of the project.


## Step-1-Sensor-Noise ##

For a kalman filter it is important to model the measurement noise matrix usually this is provided by the sensor manufacturer. In this task the drone is kept stationary in a known position and the measurements are taken using the GPS and IMU sensors to find their standard deviations. The simulation data is logged into txt files(\logs\Graph1.txt and \logs\Graph2.txt). These files are imported in a python program and their mean and standard devation are calculated using the numpy functions np.mean() and np.std(). The calculated standard devation are entered in the configuration file for this scenario. The simulator reads these values and checks if the sensor measurements lie within this range for at least ~68% of the time which is the probablity for the range $\mu\pm\sigma$ in the normal distribution. The calculated standard deviation passed this test scenario. The following picture shows the result

<img src="images\SensorNoise.png" width = "600" height = "600">


## Step-2-Attitude-Estimation ##

The starter code of the project already contains the implementation of the complimentry filter which estimates the drone's pitch, roll, and yaw angles based on the accelerometer and gyroscope measurements. But this implentation performs estimates the new drone attitude performing integration using the angular velocities from gyroscopic measurements directly. The gyroscopic measurements are in the body frame whereas the drone's attitude is in inertial frame and so this estimation works only for small angles of $\phi,\theta$. The task for this step is to improve the accurarcy of the roll and pitch angle estimation by converting the angular velocites from the gyro to inertial frame before performing the integration. This is done using the existing quaternion class. The quaternion class has a function called "FromEuler123_RPY($\phi,\theta,\psi$)" which takes the current drone's attitude and converts it into a quaternion object. The Quaternion class also has a function "IntegrateBodyRate(gyro, dt)" which perform integration when gyro measurement and sample time are given. This function performs integration using quaternion instead of Roll,Pitch and Yaw angles. Finally the Quaternion class has functions "Pitch()","Roll()","Yaw()" to return the updated roll,pitch and yaw angles. 

The test scenario measures the error of the attitutde estimation and checks if it meets the required accuracy. This implementation acheives the required result and it is shown in the following picture.

<img src="images\AttitudeEstimation.png" width = "600" height = "600">

## Step-3-Prediction-Step ##

The prediction step of the Extendend Kalman Filter contains the following steps.
 - Calculation of the new state for a given control input, current state and sample time. This is done using the state transition function $g(x_t, u_t, \Delta t)$.
 
 - Since the state transition function is non-linear it has to be differentiated to linearize the state transition function for each prediction step. This is done by calculating Jacobian matrix, which is the partical derivative of state vector of size(n) by each state variable yeilding the linear state transition matrix $g'(x_t, u_t, \Delta t)$ of size(n,n)
 
 - Then the linear state transition matrix is used to update the process covariance matrix. 

The equation of the state transition function and jacobian matrix is given in the [Estimation for Quadrotors](https://www.overleaf.com/read/vymfngphcccj#/54894644/) document. This is implemented in the project in the following functions.

- VectorXf QuadEstimatorEKF::PredictState(VectorXf curState, float dt, V3F accel, V3F gyro), this function updates the ekf state vector. 
- MatrixXf QuadEstimatorEKF::GetRbgPrime(float roll, float pitch, float yaw), this function calculates the partial derivative of rotation matrix with respect to Yaw angle. This is require to  calculate the linearized state transition matrix. This is requrired because the acceration measurements from the IMU are used are control inputs which are measured in body frame.
- void QuadEstimatorEKF::Predict(float dt, V3F accel, V3F gyro). This function calculates the linearized state transition matrix $g'$ and updates the covariance matrix.

This step has two test scenarios predict state and predict covariance. The predict state test can be performed after updating the PredictState() funciton. From this scenario it can be seen the that the IMU sensor is able to track changes in the position and velocity of the drone but it has sensor drift because accumulating the noise because of integration. This can be seen in the following picture.

<img src="images\PredictPosition.PNG" width = "600" height = "600">

The test scenario shows the position and velocity measurements for 10 steps when the drone is in stationary position. This scenario is used to tune the inital process covariance parameter since it is known that the estimation using accelerometer data has sensor drift due to the integration steps. The following picture shows this.

<img src="images\PredictCov.PNG" width = "600" height = "600">


This simulator does not measure the performance of the estimator for this step.


## Step-4-Magnetometer-Update ##


After the prediction step the update step has to be implemented. In this estimator the update step is straight forward because the IMU sensors measurement is not used in the update step which makes the update step linear. The magnetometer measures the heading of the drone by measuring the earth's magnetic field. In the update step the measurement transition function is calculated which converts the current state vector into measurement vector. In case of magnetometer the yaw angle is calulated directly it measurement transition function is vector given by $h' = [0,0,0,0,0,0,1]$. Multiplying this matrix to the state vector extract the yaw angle in the state which is the measurement of the magnetometer. 

This is implemented in the void QuadEstimatorEKF::UpdateFromMag(float magYaw) funciton. This function specifies the $h'$ matrix, calculates the yaw angle from the current state. Then it calls the function Update(z,h',R_Mag,zFromX) where the generic update equations of the kalman filter are implemented. The Update function calculates the new state and new covariance matrix after update step.

The Yaw angle from the current state has to be normalized to keep it in the range $-\pi to \pi$ before prediction step is performed. This is done by monitoring the difference between the measured yaw rate and the yaw rate of the state. The simulator has a test scenario for the yaw angle estimation.



